#Importing Data

In [0]:
from sklearn.datasets import load_digits
digits = load_digits()

#Data Grooming

##Data scaling using MinMaxScaler

In [0]:
import numpy as np
sample_x = np.random.randn(50, 5)
sample_y = np.random.randn(50, 1)
sample_x.shape, sample_y.shape

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(digits.data)
y = digits.target.reshape(-1,1)


# X = scaler.fit_transform(sample_x)
# y = sample_y

##One Hot Encoding

In [0]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=True)
y_encoded = encoder.fit_transform(y)

##Train-Test Split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Create Neural Network

##Setup Inputs

In [0]:
import numpy as np

input_size = X.shape[1]
hidden_size = input_size
output_size = len(np.unique(y))
loss_func = 'categorical_crossentropy'
epochs = 2
learning_rate = 0.1

print(f"""
        Setup details -
           input_size: {input_size}
           hidden_size: {hidden_size}
           output_size: {output_size}
           loss_func: {loss_func}
           epochs: {epochs}
           learning_rate: {learning_rate}
              """)

##Model Training

In [0]:
class NeuralNetwork():
    def __init__(self, input_size, hidden_size, output_size, loss_func):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.loss_func = loss_func

        self.weights1 = np.random.randn(self.input_size, self.hidden_size)
        self.bias1 = np.zeros((1, self.hidden_size))
        self.weights2 = np.random.randn(self.input_size, self.hidden_size)
        self.bias2 = np.zeros((1, self.hidden_size))

        print(f"""{'#'*20}
        Init w/b details -
           weights1: {self.weights1}
           bias1: {self.bias1}
           weights2: {self.weights2}
           bias2: {self.bias2}
              """)

        self.train_loss = []
        self.test_loss = []

    def __str__(self):
        return f"""{'#'*20}
        Neural Network details -
           Input Size: {self.input_size} neurons
           Hidden Size: {self.hidden_size} neurons 
           Output Size: {self.output_size} neurons 
           Loss Function: {self.loss_func} 
        """

    def forward(self, X):
        self.z1 = np.dot(X, self.weights1) + self.bias1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        if self.loss_func == 'categorical_crossentropy':
            self.a2 = self.softmax(self.z2)
        else:
            self.a2 = self.sigmoid(self.z2)
        
        print(f"""{'#'*20}
        Forward details -
           z1: {self.z1}
           a1: {self.a1}
           z2: {self.z2}
           a2: {self.a2}
              """)
        return self.a2
    
    def backward(self, X, y, learning_rate):
        m = X.shape[0]
        self.dz2 = self.a2 - y
        self.dw2 = (1/m) * np.dot(self.a2.T, self.dz2)
        self.db2 = (1/m) * np.sum(self.dz2, keepdims=True, axis=0)
        self.dz1 = np.dot(self.dz2, self.weights2.T) * self.sigmoid_derivative(self.a1)
        self.dw1 = (1 / m) * np.dot(X.T, self.dz1)
        self.db1 = (1 / m) * np.sum(self.dz1, axis=0, keepdims=True)

        self.weights2 -= learning_rate * self.dw2
        self.bias2 -= learning_rate * self.db2
        self.weights1 -= learning_rate * self.dw1
        self.bias1 -= learning_rate * self.db1
        
        print(f"""{'#'*20}
        backward details -
           dz2: {self.dz2}
           dw2: {self.dw2}
           db2: {self.db2}
           dz1: {self.dz1}
           dw1: {self.dw1}
           db1: {self.db1}

           weights1: {self.weights1}
           bias1: {self.bias1}
           weights2: {self.weights2}
           bias2: {self.bias2}
              """)
        
    def sigmoid(self, x):
        return 1 + (1/1+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps/np.sum(exps, axis=1, keepdims=True)

class Trainer():

    def __init__(self, model, loss_func):
        self.model = model
        self.loss_func = loss_func
        self.train_loss = []
        self.test_loss = []

    def calculate_loss(self, y_true, y_pred):

        if self.loss_func == 'mse':
            return np.mean((y_pred - y_true)**2)
        elif self.loss_func == 'categorical_crossentropy':
            return -np.mean(y_true * np.log(y_pred))
        else:
            raise('Invalid Loss Function used')
    
    def train(self, X_train, X_test, y_train, y_test, epochs, learning_rate):

        for _ in range(epochs):
            self.model.forward(X_train)
            self.model.backward(X_train, y_train, learning_rate)
            train_loss = self.calculate_loss(y_train, self.model.a2)
            self.train_loss.append(train_loss)

            self.model.forward(X_test)
            self.model.backward(X_test, y_test, learning_rate)
            test_loss = self.calculate_loss(y_test, self.model.a2)
            
            print(f"""{'#'*20}
            final w/b details -
            weights1: {self.model.weights1}
            bias1: {self.model.bias1}
            weights2: {self.model.weights2}
            bias2: {self.model.bias2}
                """)



In [0]:
nn = NeuralNetwork(input_size, hidden_size, output_size, loss_func)
trainer = Trainer(nn, loss_func)
trainer.train(X_train, X_test, y_train, y_test, epochs, learning_rate)

##Prediction

In [0]:
predictions = np.argmax(nn.forward(X_test), axis=1)

##Evaluation

In [0]:
accuracy = np.mean(predictions == y_test_labels)
print(f'Accuracy: {accuracy:.2%}')

#Fine Tuning the model

##Create Trial function

In [0]:
def objective(trial):
    hidden_size = trial.suggest_int('hidden_size', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    epochs = trial.suggest_int('epochs', 500, 10000)

    nn = NeuralNetwork(input_size, hidden_size, output_size, loss_func)
    trainer = Trainer(nn, loss_func)
    trainer.train(X_train, y_train, X_test, y_test, epochs, learning_rate)

    predictions = np.argmax(nn.forward(X_test), axis=1)
    accuracy = np.mean(predictions == y_test_labels)

    return accuracy

## Optimize Model

In [0]:
import optuna 

study = optuna.create_study(study_name='nn_study', direction='maximize')
study.optimize(objective, n_trials=10)

print(f"Best trial: {study.best_trial.params}")
print(f"Best value: {study.best_trial.value}")

##Best Case run

In [0]:
best_nn = NeuralNetwork(input_size, study.best_trial.params['hidden_size'], output_size, loss_func)
best_trainer = Trainer(best_nn, loss_func)
best_trainer.train(X_train, y_train, X_test, y_test, study.best_trial.params['epochs'], study.best_trial.params['learning_rate'])

##Prediction

In [0]:
predictions = np.argmax(best_nn.forward(X_test), axis=1)
accuracy = np.mean(predictions == y_test_labels)
print(f"Best accuracy: {accuracy:.2%}")